In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,038 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:1

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-26 15:45:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.95MB/s    in 0.2s    

2022-11-26 15:45:52 (5.95 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("final-project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://flightsproject.s3.us-east-2.amazonaws.com/flights_test.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("flights_test.csv"), sep=",", header=True, inferSchema=True)
#df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("flights_test.csv"), sep="\t", header=True, inferSchema=True)
df.show()


# Read in data from S3 Buckets
#from pyspark import SparkFiles
#url ="https://paolozam01-module16-practice.s3.amazonaws.com/user_data.csv"
#spark.sparkContext.addFile(url)
#user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

+---+----+-----+----------+---------+------------------+------+----+----------+-------+---------------+----------+-------+---------+----------------+--------------+-----------------+------------+------------+--------+-------------+-----------------+------------+
|_c0|Year|Month|DayofMonth|DayOfWeek|Operating_Airline |Origin|Dest|CRSDepTime|DepTime|DepDelayMinutes|CRSArrTime|ArrTime|Cancelled|CancellationCode|CRSElapsedTime|ActualElapsedTime|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|FirstDepTime|
+---+----+-----+----------+---------+------------------+------+----+----------+-------+---------------+----------+-------+---------+----------------+--------------+-----------------+------------+------------+--------+-------------+-----------------+------------+
|  0|2021|    1|         2|        6|                9E|   CVG| BOS|       730|  733.0|            3.0|       939|  927.0|      0.0|            null|         129.0|            114.0|        null|        null|   

### Create DataFrames to match tables

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [5]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://flights.cqlftqf6eowc.us-east-2.rds.amazonaws.com:5432/chall"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [6]:
# Write review_id_df to table in RDS
df.write.jdbc(url=jdbc_url, table='flights', mode=mode, properties=config)
#"review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)